In [36]:
import pandas as pd
import numpy as np

In [37]:
def enter_trade(n, prices_dataframe, Lambda):
    # calculate alpha ratio
    prices_dataframe['alpha'] = prices_dataframe['ko_Close'] / prices_dataframe['pep_Close']

    # calculate z_score
    def zscore(n):
        df_mean = prices_dataframe['alpha'].rolling(n, center=False).mean().shift(1)
        df_std = prices_dataframe['alpha'].rolling(n, center=False).std(ddof=1).shift(1)
        z_score = (prices_dataframe['alpha'] - df_mean) / df_std
        return z_score

    z_score = zscore(n)
    prices_dataframe['z_score'] = z_score

    # determine entry status for Y(ko)
    conditions = [
        (prices_dataframe['z_score'] > Lambda),
        (prices_dataframe['z_score'] <= Lambda) & (prices_dataframe['z_score'] >= -Lambda),
        (prices_dataframe['z_score'] < -Lambda)
    ]
    values = ['sell', 'do not enter', 'buy']
    prices_dataframe['action_y'] = np.select(conditions, values)
    prices_dataframe['date'] = prices_dataframe['Date'].shift(-1)
    prices_dataframe['price_y'] = prices_dataframe['ko_Open'].shift(-1)
    prices_dataframe['amount_y'] = 100  #####default enter amount is 100
    prices_dataframe['ticker_y'] = 'ko'

    # determine entry status for X(pep)
    conditions = [
        (prices_dataframe['action_y'] == 'sell'),
        (prices_dataframe['action_y'] == 'do not enter'),
        (prices_dataframe['action_y'] == 'buy')
    ]
    values = ['buy', 'do not enter', 'sell']
    prices_dataframe['action_x'] = np.select(conditions, values)
    prices_dataframe['price_x'] = prices_dataframe['pep_Open'].shift(-1)
    prices_dataframe['amount_x'] = prices_dataframe["price_y"] * prices_dataframe['amount_y'] / prices_dataframe[
        'price_x']
    prices_dataframe['ticker_x'] = 'pep'

    # create dataframe for Y(ko)
    df1 = prices_dataframe[['date', 'ticker_y', 'price_y', 'amount_y', 'action_y']]
    df_y = df1[(df1.action_y != '0') & (df1.action_y != 'do not enter')]
    df_y.rename(columns={'ticker_y': 'ticker', 'price_y': 'price', 'amount_y': 'quantity', 'action_y': 'action'},
                inplace=True)

    # create dataframe for X(pep)
    df2 = prices_dataframe[['date', 'ticker_x', 'price_x', 'amount_x', 'action_x']]
    df_x = df2[(df2.action_x != '0') & (df2.action_x != 'do not enter')]
    df_x.rename(columns={'ticker_x': 'ticker', 'price_x': 'price', 'amount_x': 'quantity', 'action_x': 'action'},
                inplace=True)

    # concat X and Y
    frames = [df_y, df_x]
    result = pd.concat(frames)
    result['trip'] = 'entry'
    result['status'] = 'FILLED'
    result['date'] = pd.to_datetime(result['date'])
    result.sort_values(by='date', inplace=True, ascending=True)
    return result

In [38]:
def exit_trade(n, prices_dataframe, entry_trade, rho, L, X):
    # copy entry_trade 作为exit修改的基础

    first_index = entry_trade.first_valid_index()
    last_index = entry_trade.index[-1]

    exit_order = entry_trade.copy(deep=True)

    # 遍历 entry_trade 修改action和trip的状态
    exit_order['trip'] = 'exit'

    # change LMT price
    price2 = [
        (exit_order['action'] == 'sell'),
        (exit_order['action'] == 'buy')
    ]
    # create a list of values we want to assign for each condition
    price2_values = [exit_order['price'] * (1 + rho), exit_order['price'] * (1 - rho)]
    # create a new column and use np.select to assign values to it using our lists as arguments
    exit_order['price_exit'] = np.select(price2, price2_values)

    # create a list of action2 to change action
    action2 = [
        (exit_order['action'] == 'sell'),
        (exit_order['action'] == 'buy')
    ]
    # create a list of values we want to assign for each condition
    action2_values = ['buy', 'sell']
    # create a new column and use np.select to assign values to it using our lists as arguments
    exit_order['action_exit'] = np.select(action2, action2_values)

    del exit_order['action'], exit_order['price']
    order = ['date', 'ticker', 'price_exit', 'quantity', 'action_exit', 'trip', 'status']
    exit_order = exit_order[order]
    exit_order.columns = ['date', 'ticker', 'price', 'quantity', 'action', 'trip', 'status']

    # 判断status
    exit_order['date_exit'] = ''
    exit_order['status_exit'] = ''

    for i in range(first_index, last_index + 1):
        if i in exit_order.index:
            if exit_order['ticker'][i].values[0] == 'ko' and exit_order['action'][i].values[0] == 'buy':
                for j in range(i + 1, i + X+1):  # x=60
                    Loss = ((prices_dataframe['ko_Close'][j] - exit_order['price'][i].values[0]) /
                            exit_order['price'][i].values[0]) * 100
                    if Loss > L:  # L=20
                        exit_order['status_exit'][i] = 'STOPLOSS'
                        exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                        break
                    elif exit_order['price'][i].values[0] > prices_dataframe['ko_Close'][j]:
                        exit_order['status_exit'][i] = 'FILLED'
                        exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                        break
                    else:
                        if i + X+1 > last_index:
                            exit_order['status_exit'][i] = 'OPEN'
                            exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                            break
                        else:
                            exit_order['status_exit'][i] = 'TIMEOUT'
                            exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                            break
            elif exit_order['ticker'][i].values[0] == 'ko' and exit_order['action'][i].values[0] == 'sell':
                for j in range(i + 1, i + X+1):
                    Loss = -((prices_dataframe['ko_Close'][j] - exit_order['price'][i].values[0]) /
                             exit_order['price'][i].values[0]) * 100
                    if Loss > L:  # L=20
                        exit_order['status_exit'][i] = 'STOPLOSS'
                        exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                        break
                    elif exit_order['price'][i].values[0] < prices_dataframe['ko_Close'][j]:
                        exit_order['status_exit'][i] = 'FILLED'
                        exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                        break
                    else:
                        if i + X+1 > last_index:
                            exit_order['status_exit'][i] = 'OPEN'
                            exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                            break
                        else:
                            exit_order['status_exit'][i] = 'TIMEOUT'
                            exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                            break
            elif exit_order['ticker'][i].values[0] == 'pep' and exit_order['action'][i].values[0] == 'buy':
                for j in range(i + 1, i + X+1):
                    Loss = ((prices_dataframe['pep_Close'][j] - exit_order['price'][i].values[0]) /
                            exit_order['price'][i].values[0]) * 100
                    if Loss > L:  # L=20
                        exit_order['status_exit'][i] = 'STOPLOSS'
                        exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                        break
                    elif exit_order['price'][i].values[0] > prices_dataframe['pep_Close'][j + 1]:
                        exit_order['status_exit'][i] = 'FILLED'
                        exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                        break
                    else:
                        if i + X+1 > last_index:
                            exit_order['status_exit'][i] = 'OPEN'
                            exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                            break
                        else:
                            exit_order['status_exit'][i] = 'TIMEOUT'
                            exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                            break
            else:
                for j in range(i + 1, i + X+1):
                    Loss = -((prices_dataframe['pep_Close'][j] - exit_order['price'][i].values[0]) /
                             exit_order['price'][i].values[0]) * 100
                    if Loss > L:  # L=20
                        exit_order['status_exit'][i] = 'STOPLOSS'
                        exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                        break
                    elif exit_order['price'][i].values[0] < prices_dataframe['pep_Close'][j + 1]:
                        exit_order['status_exit'][i] = 'FILLED'
                        exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                        break
                    else:
                        if i + X+1 > last_index:
                            exit_order['status_exit'][i] = 'OPEN'
                            exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                            break
                        else:
                            exit_order['status_exit'][i] = 'TIMEOUT'
                            exit_order['date_exit'][i] = prices_dataframe['Date'][j + 1]
                            break
        else:
            continue

    # format the dataframe
    del exit_order['date'], exit_order['status']
    order = ['date_exit', 'ticker', 'price', 'quantity', 'action', 'trip', 'status_exit']
    exit_order = exit_order[order]
    exit_order.columns = ['date', 'ticker', 'price', 'quantity', 'action', 'trip', 'status']
    exit_order['date'] = pd.to_datetime(exit_order['date'])

    # concat X and Y
    frames = [exit_order, entry_trade]
    result = pd.concat(frames)
    result['date'] = pd.to_datetime(result['date'])
    result.sort_values(by='date', inplace=True, ascending=True)
    return result

In [39]:
csv_file = "worriors_data_raw.csv"
csv_data = pd.read_csv(csv_file, low_memory=False)  # 防止弹出警告
csv_df = pd.DataFrame(csv_data)
del csv_df['Unnamed: 9'], csv_df['Unnamed: 10'], csv_df['Unnamed: 11'], csv_df['Unnamed: 12']
result = enter_trade(60, csv_df, 1)
exit_result = exit_trade(60, csv_df, result, 0.02, 20, 60)

/var/folders/q5/8pth12v54gs74jcrx3l_hx540000gn/T/ipykernel_76649/2184084346.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_y.rename(columns={'ticker_y': 'ticker', 'price_y': 'price', 'amount_y': 'quantity', 'action_y': 'action'},
/var/folders/q5/8pth12v54gs74jcrx3l_hx540000gn/T/ipykernel_76649/2184084346.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_x.rename(columns={'ticker_x': 'ticker', 'price_x': 'price', 'amount_x': 'quantity', 'action_x': 'action'},
/var/folders/q5/8pth12v54gs74jcrx3l_hx540000gn/T/ipykernel_76649/3005774497.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [40]:
exit_result

,date,ticker,price,quantity,action,trip,status
83,2017-05-04,pep,112.1700,38.709102,buy,entry,FILLED
83,2017-05-04,ko,43.4200,100.000000,sell,entry,FILLED
84,2017-05-05,ko,43.7900,100.000000,sell,entry,FILLED
84,2017-05-05,pep,113.0400,38.738500,buy,entry,FILLED
83,2017-05-05,ko,44.2884,100.000000,buy,exit,FILLED
...,...,...,...,...,...,...,...
1310,2022-03-21,ko,61.5060,100.000000,buy,exit,FILLED
1312,2022-03-22,pep,162.8400,37.417097,buy,entry,FILLED
1312,2022-03-22,ko,60.9300,100.000000,sell,entry,FILLED
1312,2022-03-23,ko,62.1486,100.000000,buy,exit,FILLED


In [41]:
exit_order_mkt = exit_result.copy(deep=True)

# select rows where status is TIMEOUT or STOPLOSS
exit_order_mkt = exit_order_mkt.loc[exit_order_mkt['status'].isin(['STOPLOSS', 'TIMEOUT'])]

exit_order_mkt

,date,ticker,price,quantity,action,trip,status
283,2018-02-21,ko,45.6450,100.000000,buy,exit,TIMEOUT
283,2018-02-21,pep,108.7408,40.329849,sell,exit,TIMEOUT
313,2018-04-05,ko,44.0028,100.000000,buy,exit,TIMEOUT
313,2018-04-05,pep,105.6244,40.025979,sell,exit,TIMEOUT
332,2018-05-02,ko,43.8192,100.000000,buy,exit,TIMEOUT
...,...,...,...,...,...,...,...
1137,2021-07-14,ko,53.8510,100.000000,sell,exit,TIMEOUT
1197,2021-10-07,ko,51.7146,100.000000,sell,exit,TIMEOUT
1197,2021-10-07,pep,153.8568,34.984089,buy,exit,TIMEOUT
1238,2021-12-06,ko,52.2634,100.000000,sell,exit,TIMEOUT


In [42]:
first_index = exit_order_mkt.first_valid_index()
last_index = exit_order_mkt.index[-1]

print(first_index)
print(last_index)

283
1238


In [43]:
exit_order_mkt['price_mkt'] = ''
exit_order_mkt['status_mkt'] = 'FILLED'

In [44]:
for i in range(first_index, last_index + 1):
    if i in exit_order_mkt.index:
        if exit_order_mkt['ticker'][i].values[0] == 'ko':
            exit_order_mkt['price_mkt'][i].values[0] = csv_df['ko_Close'][i + 2]
            exit_order_mkt['price_mkt'][i].values[1] = csv_df['pep_Close'][i + 2]
        else:
            exit_order_mkt['price_mkt'][i].values[0] = csv_df['pep_Close'][i + 2]
            exit_order_mkt['price_mkt'][i].values[1] = csv_df['ko_Close'][i + 2]
    else:
        continue

In [45]:
exit_order_mkt

,date,ticker,price,quantity,action,trip,status,price_mkt,status_mkt
283,2018-02-21,ko,45.6450,100.000000,buy,exit,TIMEOUT,43.34,FILLED
283,2018-02-21,pep,108.7408,40.329849,sell,exit,TIMEOUT,108.0,FILLED
313,2018-04-05,ko,44.0028,100.000000,buy,exit,TIMEOUT,44.4,FILLED
313,2018-04-05,pep,105.6244,40.025979,sell,exit,TIMEOUT,110.57,FILLED
332,2018-05-02,ko,43.8192,100.000000,buy,exit,TIMEOUT,42.06,FILLED
...,...,...,...,...,...,...,...,...,...
1137,2021-07-14,ko,53.8510,100.000000,sell,exit,TIMEOUT,56.26,FILLED
1197,2021-10-07,ko,51.7146,100.000000,sell,exit,TIMEOUT,53.88,FILLED
1197,2021-10-07,pep,153.8568,34.984089,buy,exit,TIMEOUT,156.39,FILLED
1238,2021-12-06,ko,52.2634,100.000000,sell,exit,TIMEOUT,54.91,FILLED


In [46]:
# format the dataframe
del exit_order_mkt['price'], exit_order_mkt['status']
order2 = ['date', 'ticker', 'price_mkt', 'quantity', 'action', 'trip', 'status_mkt']
exit_order_mkt = exit_order_mkt[order2]
exit_order_mkt.columns = ['date', 'ticker', 'price', 'quantity', 'action', 'trip', 'status']
exit_order_mkt['date'] = pd.to_datetime(exit_order_mkt['date'])

exit_order_mkt

,date,ticker,price,quantity,action,trip,status
283,2018-02-21,ko,43.34,100.000000,buy,exit,FILLED
283,2018-02-21,pep,108.0,40.329849,sell,exit,FILLED
313,2018-04-05,ko,44.4,100.000000,buy,exit,FILLED
313,2018-04-05,pep,110.57,40.025979,sell,exit,FILLED
332,2018-05-02,ko,42.06,100.000000,buy,exit,FILLED
...,...,...,...,...,...,...,...
1137,2021-07-14,ko,56.26,100.000000,sell,exit,FILLED
1197,2021-10-07,ko,53.88,100.000000,sell,exit,FILLED
1197,2021-10-07,pep,156.39,34.984089,buy,exit,FILLED
1238,2021-12-06,ko,54.91,100.000000,sell,exit,FILLED


In [47]:
frames = [exit_result, exit_order_mkt]
result = pd.concat(frames)
result['date'] = pd.to_datetime(result['date'])
result.sort_values(by='date', inplace=True, ascending=True)

result

,date,ticker,price,quantity,action,trip,status
83,2017-05-04,pep,112.17,38.709102,buy,entry,FILLED
83,2017-05-04,ko,43.42,100.000000,sell,entry,FILLED
84,2017-05-05,ko,43.79,100.000000,sell,entry,FILLED
84,2017-05-05,pep,113.04,38.738500,buy,entry,FILLED
83,2017-05-05,ko,44.2884,100.000000,buy,exit,FILLED
...,...,...,...,...,...,...,...
1310,2022-03-21,ko,61.506,100.000000,buy,exit,FILLED
1312,2022-03-22,pep,162.84,37.417097,buy,entry,FILLED
1312,2022-03-22,ko,60.93,100.000000,sell,entry,FILLED
1312,2022-03-23,pep,159.5832,37.417097,sell,exit,FILLED
